# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
file = 'City_Weather_Data.csv'
city_df = pd.read_csv(file, encoding="ISO-8859-1")
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carnarvon,-24.8667,113.6333,298.15,100,40,3.60,AU,1613163872
1,1,upernavik,72.7868,-56.1549,255.14,80,0,3.04,GL,1613163651
2,2,zyryanka,65.7500,150.8500,239.50,85,95,1.77,RU,1613163794
3,3,severo-kurilsk,50.6789,156.1250,270.47,95,80,4.80,RU,1613163869
4,4,cape town,-33.9258,18.4232,294.82,83,0,7.72,ZA,1613163482


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=g_key)
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"].astype(int)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:
m = (city_df['Max Temp'] <= 299) & (city_df['Max Temp'] >= 294) & (city_df['Wind Speed'] <= 10)\
                            & (city_df['Cloudiness'] == 0)
hotel_df = city_df.loc[m,:]
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,cape town,-33.9258,18.4232,294.82,83,0,7.72,ZA,1613163482
17,17,ancud,-41.8697,-73.8203,294.15,49,0,3.09,CL,1613163738
32,32,kaeo,-35.1000,173.7833,294.26,69,0,1.79,NZ,1613163878
37,37,coihaique,-45.5752,-72.0662,298.15,25,0,5.14,CL,1613163780
74,74,bredasdorp,-34.5322,20.0403,295.15,73,0,2.06,ZA,1613163756
87,87,arlit,18.7369,7.3853,296.63,17,0,5.10,NE,1613163889
90,90,hermanus,-34.4187,19.2345,295.37,82,0,0.45,ZA,1613163761
151,151,salalah,17.0151,54.0924,295.15,49,0,2.06,OM,1613163690
156,156,sao filipe,14.8961,-24.4956,295.88,64,0,8.80,CV,1613163902
264,264,tombouctou,20.0000,-3.0000,298.30,13,0,4.20,ML,1613163926


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [90]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "radius" : 5000,
          "key" : g_key}
for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]

    params["location"] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:

        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df



Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,cape town,-33.9258,18.4232,294.82,83,0,7.72,ZA,1613163482,Cape Town
17,17,ancud,-41.8697,-73.8203,294.15,49,0,3.09,CL,1613163738,Ancud
32,32,kaeo,-35.1000,173.7833,294.26,69,0,1.79,NZ,1613163878,Kaeo
37,37,coihaique,-45.5752,-72.0662,298.15,25,0,5.14,CL,1613163780,Coyhaique
74,74,bredasdorp,-34.5322,20.0403,295.15,73,0,2.06,ZA,1613163756,Bredasdorp
87,87,arlit,18.7369,7.3853,296.63,17,0,5.10,NE,1613163889,Arlit
90,90,hermanus,-34.4187,19.2345,295.37,82,0,0.45,ZA,1613163761,Hermanus
151,151,salalah,17.0151,54.0924,295.15,49,0,2.06,OM,1613163690,Salalah
156,156,sao filipe,14.8961,-24.4956,295.88,64,0,8.80,CV,1613163902,Sao Filipe
264,264,tombouctou,20.0000,-3.0000,298.30,13,0,4.20,ML,1613163926,


In [84]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [91]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure

fig

Figure(layout=FigureLayout(height='420px'))